# Personal Protective Equipment Dataset Explorative Data Analysis

## Imports and data loading

### Imports and paths

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
train_path = '/dataset/train'

In [4]:
print(os.getcwd())
os.chdir('..')
print(os.getcwd())

/Users/pawelgrabinski/Documents/Pet/surveily/surveily
/Users/pawelgrabinski/Documents/Pet/surveily


### Loading image data

In [ ]:
image_data = []
path_to_train_images = os.path.join(os.getcwd(), 'dataset/train/images')
for image_path in os.listdir(path_to_train_images):
    image_size = cv2.imread(os.path.join(path_to_train_images, image_path)).shape
    image_data.append((image_path, image_size))

In [ ]:
image_df = pd.DataFrame(image_data, columns=['image_name', 'dimensions'])

In [ ]:
image_df = image_df.assign(height=image_df.dimensions.apply(lambda x: x[0]),
                           width=image_df.dimensions.apply(lambda x: x[1]))

In [ ]:
image_df = image_df.assign(dimensions=image_df.height.apply(str) + 'x' + image_df.width.apply(str))

In [ ]:
image_df = image_df.assign(image_name=image_df.image_name.apply(lambda x: x[:-4]))

### Loading labels data

In [ ]:
label_data = []
path_to_train_labels = os.path.join(os.getcwd(), 'dataset/train/labels')
for labels_path in os.listdir(path_to_train_labels):
    with open(os.path.join(path_to_train_labels, labels_path)) as file:
        labels = file.readlines()
    labels = [line.strip().split() for line in labels]
    labels = [[labels_path.replace('.txt',''),] + line for line in labels]
    label_data.extend(labels)

In [ ]:
labels_df = pd.DataFrame(label_data, columns=['image_name', 'class', 'x_box_center', 'y_box_center', 'box_width', 'box_height'])

### Merging data

In [ ]:
full_df = labels_df.set_index('image_name').join(image_df.set_index('image_name'), on='image_name', how='left', lsuffix='_labels', rsuffix='_image', validate='many_to_one')

In [ ]:
full_df = full_df.reset_index()

In [ ]:
full_df = full_df.assign()

In [ ]:
full_df = full_df.assign(**{
    column: full_df.loc[:, column].astype(float) for column in [
        'x_box_center', 'y_box_center', 'box_width', 'box_height']
    })

In [ ]:
full_df = full_df.assign(**{
    column: full_df.loc[:, column].astype(int) for column in ['class', 'height', 'width']
    })

In [ ]:
full_df = full_df.assign(aspect=lambda x: x.width/x.height)

## Explorative data analysis

In [ ]:
full_df

### Balans klas

In [ ]:
pd.concat([full_df.value_counts('class'), full_df.value_counts('class', normalize=True)], axis=1)

In [ ]:
full_df.value_counts('class', normalize=True)[:4].sum()

Na początek możemy zauważyć, że zbiór jest zdecydowanie niezbalansowany pod względem klas. Klasa o indeksie `5` stanowi prawie jedną czwartą wszystkich etykiet. Mamy 3 klasy o podobnej liczebności, ale kolejne klasy są już kilkukrotnie mniej liczna, a najmniejliczne klasa stanowi niecały procent obserwacji.

Póki co na tej podstawie możemy powiedzieć, że zbiór nie nadaje się do uczenia modelu bez dodatkowych kroków. Większość algorytmów podczas uczenia będzie zwracać wyniki skupione głównie na 4 najliczniejszych klasach, które stanowią ponad `77%` obserwacji w zbiorze.

By skorygować te problemy można podejść do problemu za pomocą algorytmów, które nie będą wrażliwe na różną liczebność klas. Na przykład możemy podejść do problemu jako klasyfikacji "multi-class multi-label", więc dla każdej z nich możemy spróbować wyuczyć osobny klasyfikator. Jednak niska liczba próbek dla najmniej licznej klasy prawdopodobnie nie pozwali na osiągnięcie zadowalających wyników.

W następnym kroku można zrezygnować z detekcji tych najmniej licznych klas.

### Wymiary i aspekt

In [ ]:
full_df.describe()

Dalej możemy skupić się na wymiarach obrazów. Widzimy, że w zbiorze znajduje się pewna liczba małych obrazów, dla których maksimum dla obu szerokości lub wysokości z występujących jest kilkukortnie większe od minimum. Jednak widzimy, że większość z nich, bo już dla percentyla `25%`, ma wymiary conajmniej 720x1280 czyli ustandaryzowana rozdzielczość HD ready. Widzimy po maksiumum też, że istnieją obrazt większe. Problem z rozmiarem polega na tym, że część modeli może oczekiwać ustandaryzowanej rozdzielczości plików, więc wymagałoby to skalowania niektórych z nich. Jeśli zdecydujemy się skalować w dół, to tracimy część informacji, a jeśli decydujemy się skalować w górę, to mniejsze obrazy nie będą zawierały odpowiednich detali, które zawierają większe obrazy.

Widzimy, że aspekt obrazów również może się znacząco różnić. To nakłada potencjalnie kolejne ograniczenie na modele, które chcemy wykorzystywać. Nawet przy założeniu skalowania części z obrazów, jeśli przeskalujemy je do innego aspektu, to cechy, które rozpoznają modele mogą zostać zdeformowane, a może mieć to istotny wpływ na ich działanie.

Spróbujmy sprawdzić, jaka część zbioru ma aspekt ten sam, co HD ready.

In [ ]:
full_df.loc[(full_df.aspect-1280/720).abs()<1e-6].image_name.count()

Czyli mniej od całego zbioru o:

In [ ]:
full_df.image_name.count() - full_df.loc[(full_df.aspect-1280/720).abs()<1e-6].image_name.count()

In [ ]:
f'Jest to około  {round((full_df.image_name.count() - full_df.loc[(full_df.aspect-1280/720).abs()<1e-6].image_name.count())/full_df.image_name.count()*100,2)}% zbioru'

Możemy rozważyć statystykę takiego zbioru.

In [ ]:
same_aspect_df = full_df.loc[(full_df.aspect-1280/720).abs()<1e-6]

In [ ]:
same_aspect_df.describe()

In [ ]:
same_aspect_df.value_counts('dimensions')

Widzimy, że zostały nam jedynie obrazy ze standardu HD ready i Full HD, gdzie tych drugich. Ponieważ HD ready zawiera już dużo szczegółów, a obrazów Full HD jest niewiele, to możemy postawić hipotezę, że można te drugie przeskalować w dół do HD ready i będziemy mieli jednolity pod względem wymiarów i tym samym aspektu zbiór obrazów.

Sprawdźmy jeszcze, jak zmieniły się liczebności klas po tym odsianiu części obserwacji.

#### Balans klas zbioru o stałym aspekcie

In [ ]:
pd.concat([same_aspect_df.value_counts('class'), same_aspect_df.value_counts('class', normalize=True)], axis=1)

Niestety widzimy, że tym samym straciliśmy zupełnie jedynastą klasę, a kolejne cztery mają liczebność mniejszą niż klasa jedynasta przed redukcją zbioru. W tym miejscu powinniśmy zadać sobie pytanie, czy klasy te są kluczowe dla rozwiązania problemu.

Jeśli są, to albo musimy poluzować założenia na temat wymiarów i aspektu albo popracować nad zbiorem danych. Możliwe, że w danych, które odrzuciliśmy są obrazy, które bez utraty zaznaczonych obiektów można dociąć do oczekiwanego przez nas aspektu. 

W przeciwnym razie konieczne będzie rozszerzenie zbioru danych kolejnymi ręcznie oznaczanymi przykładami.

### Zaznaczenia

Sprawdźmy zaznaczenia i czy ich rozkłady zmieniły się po ograniczeniu zbioru.
Na początek cały zbiór.

In [ ]:
full_df.loc[:, ['x_box_center', 'y_box_center', 'box_width', 'box_height']].hist(bins=30)
plt.show()

In [ ]:
same_aspect_df.loc[:, ['x_box_center', 'y_box_center', 'box_width', 'box_height']].hist(bins=30)
plt.show()

Widzmimy, że rozkłady nie różnią się znacznie. Dla współrzędnych środka widzimy, że zaznaczenia rzadko pojawiają się na środku, ale za to mają dwie mody każda skoncentrowana wokół 0.25 i 0.75 względnego rozmiaru obrazu.

Rozmiary zaznaczeń są w większości znacznie mniejsze od rozmiaru samego obrazu, więc jest szansa, że rzeczywiście te z innymi aspektami dałoby się dosztukować do aspektu HD.

Widzimy także pewne artefakty na tych histogramach, które prawdopodobnie świadczą o tym, że znaczna część zaznaczeń wraz z odpowiadającymi im obrazami pochodzi z tego podobnego procesu, gdzie albo obiekty występują w konkretnym miejscu, albo jest to seria klatek z okresu o niedużej zmienności przedstawianego otoczenia.

In [ ]:
full_df.x_box_center.apply(lambda x: round(x, 2)).value_counts()

Widzmimy faktycznie dużo próbek dla wartości współrzędnej poziomej o wartości 0.41-0.43.

In [ ]:
full_df.loc[(full_df.x_box_center <0.44) & (full_df.x_box_center > 0.4)].image_name.apply(lambda x: x.split('_')[0][:5]).value_counts()

Widzimy po nazwach plików, że faktycznie znaczna część z obrazów, które mają te wartości położeń zaznaczeń, ma nazwy zaczynające się podobnie. Nadmierna liczba próbek z tego samego procesu może mieć silną korelację, co może prowadzić do problemów z generalizacją modeli, które nauczą się cech tła zamiast cech obiektów, które mają wykrywać.

#### Liczba zanzaczeń

Możemy jeszcze zobaczyć, jak rozkłada się liczba zaznaczeń per obraz w obu wersjach zbioru.

In [ ]:
full_df.image_name.value_counts().value_counts().plot(style='.')
plt.xticks(range(int(1), 14))
plt.show()

In [ ]:
same_aspect_df.image_name.value_counts().value_counts().plot(style='.')
plt.xticks(range(int(1), 14))
plt.show()

Widzimy, że większość obrazów rzeczywiście ma niewiele zaznaczeń 1-5. Co ciekawe odsiewając obrazy o innych aspektach straciliśmy te, które miały bardzo duże liczby zaznaczeń, co może być przypadkiem brzegowym, który i tak byłby trudno dla modelu do rozpoznania.

## Podsumowanie

1. Głównym problemem zbioru danych jest niezbalansowanie klas.
2. Część obrazów jest w nietypowej rozdzielczości. Mają inne aspekt albo bardzo małe lub duże wymiary.
3. Po odsianiu nietypowych obrazów niezbalansowanie klas się pogłębia.

  #### Co można zrobić?
1. Trzeba przejrzeć dostępne modele i zrewidować, czy problem balansu klas, wymiarów lub aspektu jest istotny.
2. Edytować nietypowe obrazy, by przy pomocy ręcznych narzędzi cięcia i skalowania ustandaryzować je.
3. W skrajnym przypadku poszerzyć zbiór o nowe ręcznie oznaczane próbki.